In [1]:
# add modules folder to Python's search path
import sys
from pathlib import Path
from os.path import dirname, realpath, abspath
script_dir = Path(abspath(''))
module_dir = str(script_dir.parent)
sys.path.insert(0, module_dir + '/modules')
sys.path.insert(0, module_dir + '/models') 
# import remaining modules
import numpy as np
import Lorenz63_xz as l63
import pandas as pd
import matplotlib.pyplot as plt
import utility as ut

In [2]:
# set random initial point, load the L96_10 model
np.random.seed(42)
model, gen_path = l63.get_model(x0=np.random.uniform(size=3), size=10, obs_gap=0.1)
data_folder = '../data'

In [3]:
# find initial points on the attractor
@ut.timer
def gen_x0(num_pts, burn_in=1000):
    pts_on_atr = np.zeros((num_pts, 3))
    X0 = np.random.uniform(size=(num_pts, 3))
    for i in range(num_pts):
        print('Working on point #{}'.format(i), end='\r')
        pts_on_atr[i, :] = gen_path(X0[i], burn_in)[-1]

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1, projection='3d')
    ax.scatter3D(pts_on_atr[:, 0], pts_on_atr[:, 1], pts_on_atr[:, 2])
    plt.savefig('{}/pts_on_attractor.png'.format(data_folder))
    np.save('{}/x0.npy'.format(data_folder), pts_on_atr)
    
# generate paths from initial points
@ut.timer
def gen_paths(length):
    X0 = np.load('{}/x0.npy'.format(data_folder))
    paths = np.zeros((len(X0), length, X0.shape[-1]))
    for i in range(num_pts):
        print('Working on path #{}'.format(i), end='\r')
        paths[i, :, :] = gen_path(X0[i], length)
    np.save('{}/true_paths_{}.npy'.format(data_folder, length), paths)
        
# generate true and observed pairs
@ut.timer
def gen_data(path_to_paths, dims = [0, 1], obs_cov=0.1, reps=10, cutoff_index=None):
    paths = np.load(path_to_paths)
    if cutoff_index is not None:
        idx = cutoff_index
    else:
        idx = paths.shape[1]
    true_paths = np.zeros((paths.shape[0] * reps, idx, paths.shape[2]))
    observations = np.zeros((paths.shape[0] * reps, idx, len(dims)))
    noise_std = np.sqrt(obs_cov)
    noise = np.random.normal(scale=noise_std, size=observations.shape)
    j = 0
    for path in paths:
        for rep in range(reps):
            true_paths[j + rep] = path[:idx]
            observations[j + rep] = path[:idx, dims] + noise[j + rep]
        j += reps
    np.save('{}/trajectories.npy'.format(data_folder), true_paths)
    np.save('{}/observations.npy'.format(data_folder), observations)

In [4]:
gen_x0(num_pts=500, burn_in=1000)

KeyboardInterrupt: 

In [ ]:
gen_paths(length=100)

In [ ]:
gen_data(path_to_paths='{}/true_paths_100.npy'.format(data_folder), dims=[0, 2], obs_cov=0.1, reps=5, cutoff_index=32)